In [0]:
from delta.tables import *

stage_table_name = "incremental_load.default.orders_stage"
target_table_name = "incremental_load.default.orders"

In [0]:
stage_df = spark.read.table(stage_table_name)
stage_df.display()

In [0]:
if not spark.catalog.tableExists(stage_table_name):
    print(f"Table {stage_table_name} does not exist.")
    print("Creating table...")
    stage_df.write.format("delta").saveAsTable(stage_table_name)
else:
    print(f"Table {stage_table_name} already exists.")
    print("Loading data...")
    target_table = DeltaTable.forName(spark, target_table_name)

    merge_condition = "stage.tracking_num = target.tracking_num"



    target_table.alias("target") \
        .merge(stage_df.alias("stage"), merge_condition) \
        .whenMatchedDelete()\
        .execute()
    
    stage_df.write.format("delta").mode("append").saveAsTable(target_table_name)

In [0]:
target_df = spark.read.table(target_table_name)
target_df.display()